# Tests de la Self-Attention

In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F

import torch.optim as optim
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

### Mecanismo de Self-Attention

La clase de autoatención implementada por nuestro modelo es la siguiente:

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, inpunt_dim, hidden_dim):
        super(SelfAttention, self).__init__()
        self.inpunt_dim = inpunt_dim
        self.hidden_dim = hidden_dim
        self.Q = nn.Linear(inpunt_dim, hidden_dim, bias=False)
        self.K = nn.Linear(inpunt_dim, hidden_dim, bias=False)
        self.V = nn.Linear(inpunt_dim, hidden_dim, bias=False)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        xq = self.Q(x)
        xk = self.K(x)
        xv = self.V(x)

        scores = torch.bmm(xq, xk.transpose(1, 2))
        scores = scores / (self.hidden_dim ** 0.5)
        mask = self._mask(scores).to(x.device)
        scores = scores + mask

        scores = self.softmax(scores)
        attention = torch.bmm(scores, xv)
        return scores, attention

    def _mask(self, x):
        mask = torch.tril(torch.ones(x.size(1), x.size(1)), diagonal=0)
        mask[mask == 0] = float('-inf')
        mask[mask == 1] = 0
        mask = mask.repeat(x.size(0), 1, 1)
        return mask

Ahora bien, esta clase está preparada para recibir un tensor de entrada, y calcular sobre ellos la atención. No obstante, se pide en la práctica comprobar que el mecanismo de atención funcione correctamente sobre las matrices Q, K y V, directamente. Para ello, se aplican unas pequeñas modificaciones a esta clase, que permiten realizar el cálculo de la atención sobre dichas matrices, sin necesidad de un tensor de entrada.

In [6]:
class SelfAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SelfAttention, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.Q = nn.Linear(input_dim, hidden_dim, bias=False)
        self.K = nn.Linear(input_dim, hidden_dim, bias=False)
        self.V = nn.Linear(input_dim, hidden_dim, bias=False)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, Q=None, K=None, V=None, x=None):
        # Se comprueba si se recibe un tensor x de entrada
        if x is not None:
            Q = self.Q(x)
            K = self.K(x)
            V = self.V(x)

        # Cuando no se recibe un tensor x de entrada, hay que añadir una dimensión para el batch
        if Q.dim() == 2:
            Q = Q.unsqueeze(0)
            K = K.unsqueeze(0)
            V = V.unsqueeze(0)

        scores = torch.matmul(Q, K.transpose(1, 2))
        scores = scores / (self.hidden_dim ** 0.5)
        mask = self._mask(scores).to(Q.device)
        scores = scores + mask

        scores = self.softmax(scores)
        attention = torch.matmul(scores, V)
        return scores.squeeze(0), attention.squeeze(0)

    def _mask(self, x):
        mask = torch.tril(torch.ones(x.size(1), x.size(1)), diagonal=0)
        mask[mask == 0] = float('-inf')
        mask[mask == 1] = 0
        mask = mask.repeat(x.size(0), 1, 1)
        return mask

Testearemos ahora la atención sobre las siguientes matrices Q, K y V, dado que ya conocemos de antemano el resultado numérico que debería obtenerse.

In [7]:
Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

In [8]:
att = SelfAttention(3, 3)
scores, attention = att(Q=Q, K=K, V=V)

In [9]:
print("Scores:\n", scores)
print("\nAttention:\n", attention)

Scores:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000, 0.0000],
        [0.3219, 0.3332, 0.3449, 0.0000],
        [0.2309, 0.2432, 0.2561, 0.2698]])

Attention:
 tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


En efecto, se obtiene el resultado deseado, por lo que la atención se ha calculado con éxito.